<a href="https://colab.research.google.com/github/Tekleab15/Context-Aware-Conversational-Search-System/blob/main/notebooks/semantic_search_implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Implmenting the semantic search and measuring the cosine similarity**

Import all required modules and mounting the drive

In [2]:
# Installing the gradio-UI to show results
# !pip install gradio

In [3]:
import numpy as np
import gradio as gr
import pandas as pd
import tensorflow as tf
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from google.colab import drive
drive.mount('/content/drive')

print("Run and mounted Successfully!")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Run and mounted Successfully!


Loading the embeddings embeded using (SBERT)

In [4]:
# Load the pre-computed embeddings
embeddings_instructions = np.load('/content/drive/My Drive/Conversational_semantic_search/embeddings_instructions.npy')
embeddings_responses = np.load('/content/drive/My Drive/Conversational_semantic_search/embeddings_responses.npy')
model = SentenceTransformer('all-mpnet-base-v2')
print("Model loaded successfully")

data_path = '/content/drive/My Drive/Conversational_semantic_search/Customer_service_preprocessed_.csv'
data_path1 = '/content/drive/My Drive/Conversational_semantic_search/Customer_service.csv'
df = pd.read_csv(data_path)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Model loaded successfully


**Search implementation without using context**

In [5]:
# Response with out using context(memory of prior queries)
def et_response_without_context(user_query):
    global model
    global embeddings_instructions
    global df
    if model is None:
        return "Error: Model not loaded. Try re-running the notebook."
    try:
        query_embedding = model.encode([user_query])
        similarities = cosine_similarity(query_embedding, embeddings_instructions)
        best_match_index = similarities.argmax()
        best_response = df.iloc[best_match_index]['response']
        return best_response
    except Exception as e:
        return f"An error occurred: {str(e)}"


In [6]:
print(et_response_without_context("How can I cancel my order?"))

Assuredly! I understand that you want to cancel the purchase you made. Let me guide you through the process step by step: 

1. Log in to your {{Online Company Portal Info}} using your credentials.
2. Navigate to the '{{Online Order Interaction}}' or '{{Online Order Interaction}}' section, where you can find a list of your previous purchases.
3. Identify the specific purchase you want to cancel. You may need to search for it by order number or date.
4. Once you've located the purchase, look for an option labeled '{{Online Order Interaction}}' or something similar. Click on it.
5. Follow the instructions provided by the system to complete the cancellation process.

If you encounter any difficulties or have any questions along the way, our dedicated customer support team is here to assist you. You can reach us during {{Customer Support Hours}} at {{Customer Support Phone Number}} or through the Live Chat on our website at {{Website URL}}. We value your satisfaction and will do our best to

**Adding Context (memory)**

In [7]:
# Initializing session memory to store the last 2-3 queries
session_memory = []

def get_response_with_context(user_query):
    global model
    global embeddings_instructions
    global df
    global session_memory

    # Check if the model is loaded
    if model is None:
        return "Error: Model not loaded. Please ensure the model is loaded before querying."
    try:
        # Add the current query to the session memory (limit to the last 4 queries)
        session_memory.append(user_query)
        # Limiting the memory size to last 4 queries
        if len(session_memory) > 4:
            session_memory.pop(0)
        # Combine all previous queries to maintain context
        context = " ".join(session_memory)
        context_embedding = model.encode([context])
        # Calculate cosine similarities between the context and embeddings
        similarities = cosine_similarity(context_embedding, embeddings_instructions)

        # Identify the best matching response based on the highest similarity
        best_match_index = similarities.argmax()
        best_response = df.iloc[best_match_index]['response']
        return best_response
    except Exception as e:
        # Handle any unexpected errors
        return f"An error occurred: {str(e)}"


In [8]:
# A function that allows the user to choose between two modes:
def chat_with_context_mode(user_query, use_context):
    if use_context:
        # Get response with context
        return get_response_with_context(user_query)
    else:
        # Get response without context
        return get_response_without_context(user_query)

# Create the Gradio interface
iface = gr.Interface(
    fn=chat_with_context_mode,
    inputs=[
        gr.Textbox(label="Your Query", placeholder="Ask something..."),
        gr.Radio(["Without Context", "With Context"], label="Response Mode")
    ],
    outputs="text",
    live=False,
    title="Context-Aware Conversational Search",
    description="This system helps answer queries with or without context. Choose the response mode!"
)
# Launch the interface
iface.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://16485577ebc463de2c.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


**Intent recognition section**

In [9]:
# !pip install transformers datasets

In [ ]:
import pandas as pd
from datasets import Dataset
from transformers import BertTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments
import torch
# Load and prepare dataset
data = pd.read_csv(data_path1)
dataset = Dataset.from_pandas(data)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Create label mapping
unique_labels = list(data['intent'].unique())
label2id = {label: i for i, label in enumerate(unique_labels)}

# Tokenize dataset
def tokenize_function(examples):
    return tokenizer(examples['instruction'], padding="max_length", truncation=True)
tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets = tokenized_datasets.rename_column("intent", "labels")

# Map string labels to integers
def cast_labels_to_int(examples):
    examples["labels"] = [label2id[label] for label in examples["labels"]]
    return examples

tokenized_datasets = tokenized_datasets.map(cast_labels_to_int, batched=True)
train_test_split = tokenized_datasets.train_test_split(test_size=0.2)
train_dataset = train_test_split['train']
test_dataset = train_test_split['test']

# Fine-tune DistilBERT model
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=len(unique_labels))
training_args = TrainingArguments(
    output_dir='./results',
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=4,
    num_train_epochs=1,
    logging_dir='./logs',
    report_to="none"
)
trainer = Trainer(model=model, args=training_args, train_dataset=train_dataset, eval_dataset=test_dataset)
trainer.train()

Map:   0%|          | 0/26872 [00:00<?, ? examples/s]

Map:   0%|          | 0/26872 [00:00<?, ? examples/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss


In [ ]:
model.save_pretrained('/content/intent_model')
tokenizer.save_pretrained('/content/intent_model')
model = DistilBertForSequenceClassification.from_pretrained('/content/intent_model')
tokenizer = BertTokenizer.from_pretrained('/content/intent_model')

# Intent prediction function
def predict_intent(user_query):
    inputs = tokenizer(user_query, return_tensors="pt", padding=True, truncation=True)
    outputs = model(**inputs)
    logits = outputs.logits
    predicted_class = torch.argmax(logits, dim=1).item()
    return unique_labels[predicted_class]

**Gradio interface**

In [ ]:
# Gradio interface
def chat_with_intent_recognition(user_query, response_mode):
    intent = predict_intent(user_query)
    response = get_response_with_context(user_query) if response_mode == "With Context" else get_response_without_context(user_query)
    return f"Intent: {intent}\nResponse: {response}"

iface = gr.Interface(
    fn=chat_with_intent_recognition,
    inputs=[gr.Textbox(label="Your Query"), gr.Radio(["Without Context", "With Context"], label="Response Mode")],
    outputs="text",
    live=True,
    title="Intent-Aware Conversational Search",
    description="Classifies query intent and provides context-aware responses."
)
iface.launch()

**Building the UI(user interface)**

In [ ]:
# Define the Gradio UI
demo = gr.Interface(
    fn=get_response,
    inputs=gr.Textbox(label="Enter your search query"),
    outputs=gr.Textbox(label="Search Results"),
    title="Context-Aware Search (Customer service or support)",
    description="""The dataset is adopted from Customer service chat in the areas of Automotive, Retail Banking, Education, Events & Ticketing,
                            Field Services, Healthcare, Hospitality, Insurance, Legal Services, Manufacturing, Media Streaming, Mrtgages & Loans,
                            Moving & Storage and Real Estate/Construction Enter a query, and the AI(Context Aware Customer service) model will return context-aware search results.""")


# Launch the UI
demo.launch(share=True)
